# PaddlePaddle 实现验证码识别
欢迎大家来到这次实验，在这次实验中我们将使用 PaddlePaddle 来搭建一个CNN卷积神经网络，用于识别验证码。

**本实验我们将：**

- 温习卷积神经网络的原理及组成

- 用paddle搭建一个基于卷积神经网络的分类器，用于识别验证码

## 1 - 知识点回顾
### 1.1 - CNN是怎么来的？
对于全连接神经网络来说，随着神经网络层数的增加，将会带来维度灾难。在图像处理中，往往把图像表示为像素的向量。以200×200的图像为例，全连接神经网络的输入层：40000维，如果隐藏层神经元数目与输入层一样，即也是40000时，那么输入层到隐含层的参数数据为40000×40000=1.6×10^9,这么多的参数使得内存、计算量巨大,训练困难。所以需要减少参数数目。

### 1.2 - CNN的结构：
下图为CNN网络的整体结构：输入层、卷积层、激活函数、池化层、全连接层。
<img src="images_jupyter/cnn_1.png" width="400"><br/>
#### Conv层（卷积层）

卷积是在原始的输入上进行特征的提取。特征提取简言之就是，在原始输入上一个小区域一个小区域进行特征的提取。卷积是一种数学计算。
<img src="images_jupyter/cnn_2.png" width="400"><br/>
上图中，左方块是输入层，尺寸为32×32的3通道图像。中间的小方块是filter，尺寸为5*5，深度为3。将输入层划分为多个区域，用filter这个固定尺寸的助手，在输入层做运算，最终得到一个深度为1的特征图。一般使用多个filter分别进行卷积，最终得到多个特征图。上图显示了使用2个、6个filter进行卷积分别得到2个和6个特征图。
<img src="images_jupyter/cnn_3.png" width="500"><br/>
直观地理解卷积，以上图为例。第一次卷积可以提取出低层次的特征。第二次卷积可以提取出中层次的特征。第三次卷积可以提取出高层次的特征。
特征是不断进行提取和压缩的，最终能得到比较高层次特征，简言之就是对原式特征一步又一步的浓缩，最终得到的特征更可靠。利用最后一层特征可以做各种任务：比如分类、回归等。
<img src="images_jupyter/conv_layer.png" width="600"><br/>
上图是一个演示卷积计算过程的示例图，假如我们的输入图像大小为$H=5,W=5,D=3$，即$5 \times 5$大小的3通道（RGB，也称作深度）彩色图像。这个示例图中包含两（用$K$表示）组卷积核，即图中滤波器$W_0$和$W_1$。在卷积计算过程中，我们通常对不同的输入通道采用不同的卷积核，比如这个图中每组卷积核包含（$D=3）$个$3 \times 3$（用$F \times F$表示）大小的卷积核。此外，还有一个步长，这个示例中卷积核在图像的水平方向（$W$方向）和垂直方向（$H$方向）的滑动步长都为2（用$S$表示）；还有一个padding（也就是填充）值，对输入图像周围各填充1（用$P$表示）个0，即图中输入层原始数据为蓝色部分，灰色部分是进行了大小为1的扩展，用0来进行扩展。经过卷积操作得到输出为$3 \times 3 \times 2$（用$H_{o} \times W_{o} \times K$表示）大小的特征图，即$3 \times 3$大小的2通道特征图，其中$H_o$计算公式为：$H_o = (H - F + 2 \times P)/S + 1$，$W_o$同理。 而输出特征图中的每个像素，是每组滤波器与输入图像每个特征图的内积再求和，再加上偏置$b_o$，偏置通常对于每个输出特征图是共享的。输出特征图$o[:,:,0]$中的最后一个$-2$计算如图5右下角公式所示。

在卷积操作中卷积核是可学习的参数，经过上面示例介绍，每层卷积的参数大小为$D \times F \times F \times K$。在多层感知器模型中，神经元通常是全部连接，参数较多。而卷积层的参数较少，这也是由卷积层的局部连接和共享权重所决定。


通过以上的卷积操作，我们可以发现卷积其实是线性操作，并具有平移不变性（shift-invariant），平移不变性即在图像每个位置执行相同的操作。卷积层的局部连接和权重共享使得需要学习的参数大大减小，这样也有利于训练较大卷积神经网络。

#### 池化层（pool）
<img src="images_jupyter/max_pooling.png" width="350px"><br/>
一般情况下，在连续的卷积层之间会周期性地插入一个池化层（也称汇聚层），其处理输入数据的准则被称为池化函数。池化函数在计算某一位置的输出时，会计算该位置相邻区域的输出的某种总体统计特征，作为网络在该位置的输出。池化层的作用是逐渐降低数据体的空间尺寸，从而减少网络中参数的数量以及耗费的计算资源，同时也能有效控制过拟合。

池化包括最大池化、平均池化、L-2范数池化等。以最大池化(Max Pooling)为例，池化层使用最大化(Max)操作，即用一定区域内输入的最大值作为该区域的输出。最大池化最常用的形式是使用尺寸为$2\times2$的滤波器、步长为2来对每个深度切片进行降采样，每个Max操作是从4个数字中取最大值（也就是在深度切片中某个的区域），这样可以将其中75%的激活信息都过滤掉，而保持数据体通道数不变,具体示例如上图所示。

## 2 - 数据集介绍

本次实验我们利用代码自己生成4位验证码，验证码图片如下：
<img src="images_jupyter/shuju_1.png" width="200"><br/>
<img src="images_jupyter/shuju_2.png" width="200"><br/>
<img src="images_jupyter/shuju_3.png" width="200"><br/>
验证码生成代码在captcha/gen/gen.py 中，我们运行gen.py文件就可以生成验证码。文档架构如下：
<img src="images_jupyter/wenjian_1.png" width="500"><br/>
gen.py文件中主要有以下两个函数：
<img src="images_jupyter/gen_4.png" width="700"><br/>
<img src="images_jupyter/gen_6.png" width="700"><br/>

## 3 - 引用库

数据集生成之后，我们就可以开始进行试验了。

首先，载入几个需要用到的库，它们分别是：

- numpy：一个python的基本库，用于科学计算
- paddle.fluid：PaddlePaddle深度学习框架fluid版本
- random：Python中的random模块用于生成随机数
- pandas：Pandas是Python中最常用的数据分析模块，拥有强大的数据读写性能，丰富的可视化工具，便捷的数据切片及索引功能
- PIL：用于处理图像数据
- logging：logging模块是Python内置的标准模块，主要用于输出运行日志，可以设置输出日志的等级、日志保存路径、日志文件回滚等。
<img src="images_jupyter/import_1.png" width="500"><br/>

In [1]:
# -*- coding: utf-8 -*-

import paddle.fluid as fluid
import numpy as np
import pandas as pd
import random
import logging
from PIL import Image

## 4 - 定义读取数据函数

在训练和测试网络模型的过程中，程序必须要加载数据，为了减轻开发者在这部分的代码编写工作，Paddlepaddle 专门定义了reader机制来帮助我们进行数据读取。

它的具体实现是使用了yield关键字。
- > 注意，yield关键字的作用和使用方法类似return关键字，不同之处在于yield关键字可以构造生成器（Generator）。
- > 虽然我们可以直接创建一个包含所有数据的列表，但是由于内存限制，我们不可能创建一个无限大的或者巨大的列表，并且很多时候在创建了一个百万数量级别的列表之后，我们却只需要用到开头的几个或几十个数据，这样造成了极大的浪费，而生成器的工作方式是在每次循环时计算下一个值，不断推算出后续的元素，不会创建完整的数据集列表，从而节约了内存使用。

我们定义一个data_reader（）函数，来实现数据加载。
<img src="images_jupyter/reader_1.png" width="700"><br/>

In [2]:
def data_reader(path, shape, batch_size=128):
    """
    Define the data reader.

    """
    df = pd.read_table(path + 'labels.txt', header=None)#read_table()读取以‘/t’分割的文件到DataFrame，无header行
    height, width = shape[1], shape[2]
    for i in range(len(df) // batch_size):               #应该用“//”，——代表整除，“/”会提示错误
        data = []
        for j in range(batch_size):
            index = random.randint(0, len(df) - 1)       #随机取一个数字并选择其对应的验证码
            img = Image.open(path + str(index) + '.png') #提取图片，调整大小，调整数值
            img = img.resize([width, height])
            img = np.multiply(img, 1 / 255.0)
            img = np.asarray(img).transpose([2, 0, 1]).astype('float32')#产生一个ndarray，转置（【0，1，2】->【2，0，1】），再转换为浮点型
            label = str(df.ix[index, 0])
            label_0 = np.array(label[0]).astype('int')
            label_1 = np.array(label[1]).astype('int')
            label_2 = np.array(label[2]).astype('int')
            label_3 = np.array(label[3]).astype('int')
            temp = (img, label_0, label_1, label_2, label_3)
            data.append(temp)

        yield data

## 5- 定义网络结构

接下来我们就要自己动手写一个CNN网络了。

同样的，为了方便后面调用，我们把CNN网络定义到一个函数中：**def cnn_net(x)**  

CNN网络结构可以做如下设计：


|索引     |  网络层   |  参数描述   |
| --- | --- | --- |
|0     |   输入层  |    x   |
|1     |  卷积层1 Conv2D   | 3x3x32     |
|2     |  卷积层2 Conv2D  | 3x3x32     |
|3     |  池化层 MaxPooling2D   | 2x2     |
|4     |  卷积层3 Conv2D   | 3x3x32     |
|5     |  卷积层4 Conv2D   | 3x3x32     |
|6     |  池化层 MaxPooling2D   | 2x2     |
|7     | dropout层 Dropout    |0.25    |
|8     | 输出层     | yp0, yp1, yp2, yp3  |

**注意！！！** 我们这里加了dropout层，dropout层有什么用呢？是的，他对减少过拟合非常有用，可以增加模型的鲁棒性

### API介绍：
 PaddlePaddle中关于CNN的API介绍：

**paddle.fluid.layers.conv2d(input, num_filters, filter_size, stride=1, padding=0, dilation=1, groups=None, param_attr=None, bias_attr=None, use_cudnn=True, act=None, name=None)** 

> **解释：** 
这是一个二维卷积操作。根据输入、滤波器（filter）、步长（stride）、填充（padding）、dilations、一组参数计算输出。输入和输出是NCHW格式，N是批尺寸，C是通道数，H是特征高度，W是特征宽度。滤波器是MCHW格式，M是输出图像通道数，C是输入图像通道数，H是滤波器高度，W是滤波器宽度。  

> **参数：** 
> - input (Variable) - 格式为[N,C,H,W]格式的输入图像
> - num_fliters (int) - 滤波器数。和输出图像通道相同
> - filter_size (int|tuple|None) - 滤波器大小。如果filter_size是一个元组，则必须包含两个整型数，（filter_size，filter_size_W）。否则，滤波器为square
> - stride (int|tuple) - 步长(stride)大小。如果步长（stride）为元组，则必须包含两个整型数，（stride_H,stride_W）。否则，stride_H = stride_W = stride。默认：stride = 1
> - padding (int|tuple) - 填充（padding）大小。如果填充（padding）为元组，则必须包含两个整型数，（padding_H,padding_W)。否则，padding_H = padding_W = padding。默认：padding = 0

> **举例：**   
> <img src="images_jupyter/API_1.png" width="700"><br/>

**paddle.fluid.layers.pool2d(input, pool_size=-1, pool_type='max', pool_stride=1, pool_padding=0, global_pooling=False, use_cudnn=True, ceil_mode=False, name=None, exclusive=True)** 

> **解释：** 池化操作，根据池化核大小 pool_size , 步长 pool_stride ，填充 pool_padding 这些参数得到输出。
输入X和输出Out是NCHW格式，N为batch尺寸，C是通道数，H是特征高度，W是特征宽度。
参数（ksize,strides,paddings）含有两个元素。这两个元素分别代表高度和宽度。输入X的大小和输出Out的大小可能不一致。
> **参数：**   
> - input (Variable) - 池化操作的输入张量。输入张量格式为NCHW，N为批尺寸，C是通道数，H是特征高度，W是特征宽度
> - pool_size (int|list|tuple) - 池化核的大小。如果它是一个元组或列表，它必须包含两个整数值， (pool_size_Height, pool_size_Width)。若为一个整数，则它的平方值将作为池化核大小，比如若pool_size=2, 则池化核大小为2x2。
> - pool_type (string) - 池化类型，可以是“max”对应max-pooling，“avg”对应average-pooling
> - pool_stride (int|list|tuple) - 池化层的步长。如果它是一个元组或列表，它将包含两个整数，(pool_stride_Height, pool_stride_Width)。否则它是一个整数的平方值。
> - pool_padding (int|list|tuple) - 填充大小。如果它是一个元组或列表，它必须包含两个整数值，(pool_padding_on_Height, pool_padding_on_Width)。否则它是一个整数的平方值。

> **举例：**   
> <img src="images_jupyter/api_2.png" width="700"><br/>

**paddle.fluid.layers.dropout(x, dropout_prob, is_test=False, seed=None, name=None, dropout_implementation='downgrade_in_infer')**
> **解释：**  这是一个dropout操作
丢弃或者保持x的每个元素独立。Dropout是一种正则化技术，通过在训练过程中阻止神经元节点间的联合适应性来减少过拟合。根据给定的丢弃概率dropout操作符随机将一些神经元输出设置为0，其他的仍保持不变。
> **参数：**   
> - dropout_prob (float)-设置为0的单元的概率

> **举例：**   
> <img src="images_jupyter/api_3.png" width="700"><br/>

为了让大家达到边学边练的目的，我们这里出一个练习，把CNN网络的搭建工作交给大家来做，大家可以根据以上我们的网络设计要求，和API提示，动手尝试**def cnn_net(x)** 函数代码的编写。

**练习1**
> **要求** 
> - 完成def cnn_net(x)代码。
> - 符合上表所列模型设计： 4 cov lays和4 outputs

In [3]:
def cnn_net(x):
    """
    Define the CNN network model.
    This model has 4 cov lays and 4 outputs.
    Each output is for one digit in the image.
    Finally we optimize the 4 output at the same time, each ouput contribute 1/4 the total loss.
    """
    ###练习1：完成def cnn_net(x)函数的代码编写
    ### 练习代码开始处 ### 

    
    
    
    
    
    
    
    
    
    
    
    
    ### 练习代码结束处 ###
    return yp0, yp1, yp2, yp3

## 6 定义训练函数 train()
接下来我们定义训练函数train()，里面含有的主要功能模块有：
> - 损失函数选择
> - 优化器配置
> - 训练场所设置

**损失函数选择：**
> - 对于分类问题常常选择交叉熵损失函数。 其形式如下： cost = paddle.layer.cross_entropy(input=y_predict, label=y_label)。
> - 当输入一个batch的数据时，损失算子的输出有多个值，每个值对应一条样本的损失，所以通常会在损失算子后面使用mean等算子，来对损失做归约。 损失规约形式如下：avg_cost = fluid.layers.mean(cost)。

**优化器配置:**
> - 参数创建完成后，我们需要定义一个优化器optimizer，为了改善模型的训练速度以及效果，学术界先后提出了很多优化算法，包括： Momentum、RMSProp、Adam 等，已经被封装在fluid内部，读者可直接调用。
> - 在这里我们尝试使用Adam来作为优化器，它的计算公式如下：（由于时间关系，公示具体的含义就不和大家讲了，有兴趣的同学可以查阅相关资料）

$$\begin{split}m(w, t) & = \beta_1 m(w, t-1) + (1 - \beta_1) \nabla Q_i(w) \\ v(w, t) & = \beta_2 v(w, t-1) + (1 - \beta_2)(\nabla Q_i(w)) ^2 \\ w & = w - \frac{\eta m(w, t)}{\sqrt{v(w,t) + \epsilon}}\end{split}$$

> - Adam是一种常用的、效果良好的自适应学习率调整优化算法，通常只需要将参数设置为beta1=0.9，beta2=0.999，epsilon=1e-08，不需要作修改即可让模型产生好的收敛效果。在 fluid 中可以使用接口 fluid.optimizer.Adam() 来创建 Adam 优化器。这个函数接受 4个参数分别是：learning_rate, beta1, beta2 和 epsilon。
> - 举例：<img src="images_jupyter/adam_1.png" width="700"><br/>

> - **练习2**  

   > 本次我们可以使用Adam优化器，这样设置参数：learning_rate=0.001
请在下面空白处补充完优化器的配置语句。优化器名称**sgd_optimizer**

**训练场所设置：**

> - 训练场所设置,也就是选择是在CPU上进行训练，还是在GPU上进行训练。在复杂量较低的时候使用 CPU 就可以完成任务，但是对于大规模计算就需要使用 GPU 训练。目前 GPU 训练都是基于 CUDA 工具之上的。  
>- 代码实现也很简单，我们使用两行代码就可以实现，如下面所示：  
> use_cuda=False 表示不使用 GPU 进行训练
<img src="images_jupyter/cuda_1.png" width="700"><br/>

> - **练习3**  
请在下面空白处补充完训练场所设置的语句。

**用双层循环组成训练模块：**
> - 以上工作做好之后，我们就可以用一个双层循环，来组成训练模块了。我们的模型训练工作主要就是从这里开始，并且循环往复不断进行的。
> - 外层循环，我们可以设定总样本训练的次数（也就是epoch的数目）。
> - 内层循环就是每次从数据读取器reader中读入数据，送入我们定义的网络结构中。然后运行执行器训练。
> - 同时训练时可以把loss值、acc值的变化实时打印出来，便于我们观察模型训练情况。
> - 在此模块中我们可以选择一个文件路径，保存我们训练中（或训练好的）模型，以便于将来用这个模型进行预测。
> - 训练模块具体代码编写如下所示：
> <img src="images_jupyter/for_1.png" width="800"><br/>


In [4]:
def train(model_path, data_path, shape):
    """
    Define the train function.
    """
    #定义输入数据（图片）的格式
    x = fluid.layers.data(name='x', shape=shape, dtype='float32')
    y0 = fluid.layers.data(name='y0', shape=[1], dtype='int')
    y1 = fluid.layers.data(name='y1', shape=[1], dtype='int')
    y2 = fluid.layers.data(name='y2', shape=[1], dtype='int')
    y3 = fluid.layers.data(name='y3', shape=[1], dtype='int')
    #调用cnn网络
    yp0, yp1, yp2, yp3 = cnn_net(x)#每个代表一个数字可能性的10*1张量。
    loss_0 = fluid.layers.cross_entropy(input=yp0, label=y0)#cross entropy(交叉熵)损失函数
    loss_1 = fluid.layers.cross_entropy(input=yp1, label=y1)
    loss_2 = fluid.layers.cross_entropy(input=yp2, label=y2)
    loss_3 = fluid.layers.cross_entropy(input=yp3, label=y3)
    loss = loss_0 + loss_1 + loss_2 + loss_3
    avg_loss = fluid.layers.mean(loss)#算得平均损失

    acc0 = fluid.layers.accuracy(input=yp0, label=y0)#使用输入和标签计算准确率。
    acc1 = fluid.layers.accuracy(input=yp1, label=y1)
    acc2 = fluid.layers.accuracy(input=yp2, label=y2)
    acc3 = fluid.layers.accuracy(input=yp3, label=y3)
    
    ###练习2：完成 优化器配置 的代码编写
    ### 练习代码开始处 ### 

    
    ### 练习代码结束处 ###

    ###练习3：完成 训练场所设置 的代码编写
    ### 练习代码开始处 ###

    
    ### 练习代码结束处 ###
    
    exe = fluid.Executor(place)                     #定义执行器

    feeder = fluid.DataFeeder(place=place, feed_list=[x, y0, y1, y2, y3]) #设定一个数据feeder
    exe.run(fluid.default_startup_program())                              #预先启动一下以用于初始化参数

    epoch_num = 1000                                                      #1个epoch等于使用训练集中的全部样本训练一次
    for epoch in range(epoch_num):
        total_loss_value = 0
        step = 0
        for data in data_reader(data_path, shape, batch_size=128):        #在此处按一个batch=128个数据读取图片，如果batch_size设置过大，GPU运行有时报错

            avg_loss_value, acc0_value, acc1_value, acc2_value, acc3_value = exe.run(
                fluid.default_main_program(), feed=feeder.feed(data), fetch_list=[avg_loss, acc0, acc1, acc2, acc3])
            total_loss_value += avg_loss_value

            flag = acc0_value > 0.99 and acc1_value > 0.99 and acc2_value > 0.99 and acc3_value > 0.99
            if flag or epoch == epoch_num - 1:#训练终止的条件——训练次数达到要求或者精度达到要求
                if model_path is not None:
                    fluid.io.save_inference_model(#保存模型用于使用
                        model_path, ['x'], [yp0, yp1, yp2, yp3], exe)
                return

            print("Epoch %d, Step %d, avg loss = %.4f, acc0 = %.2f, acc1 = %.2f, acc2 = %.2f, acc3 = %.2f" %
                         (epoch, step, avg_loss_value, acc0_value, acc1_value, acc2_value, acc3_value))            
            
            step += 1
        print("Epoch %d, total loss = %.4f" % (epoch, total_loss_value))

## 7 开始训练
> 前面的工作完成之后，我们就可以设置参数，调用train()函数，来开始模型的训练了。
> 我们需要设置的参数包括：
> <img src="images_jupyter/train_1.png" width="600"><br/>

> 训练过程中，loss、acc的实时输出是这样的：
> <img src="images_jupyter/train_2.png" width="600"><br/>


In [5]:
model_path = "capture.inference.model"
train_path = '../gen/4_train/'
test_path = '../gen/4_test/'
width = 160
height = 60
channel = 3
image_shape = [channel, height, width]#定义图片大小及通道数，这需要根据生成的验证码实际情况而定
train(model_path, train_path, image_shape) #训练神经网络



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


Epoch 0, Step 0, avg loss = 9.8769, acc0 = 0.14, acc1 = 0.10, acc2 = 0.11, acc3 = 0.11
Epoch 0, Step 1, avg loss = 153.1013, acc0 = 0.10, acc1 = 0.09, acc2 = 0.12, acc3 = 0.09
Epoch 0, Step 2, avg loss = 188.2334, acc0 = 0.08, acc1 = 0.09, acc2 = 0.05, acc3 = 0.05
Epoch 0, Step 3, avg loss = 158.1059, acc0 = 0.16, acc1 = 0.15, acc2 = 0.08, acc3 = 0.11
Epoch 0, Step 4, avg loss = 92.8917, acc0 = 0.06, acc1 = 0.07, acc2 = 0.09, acc3 = 0.11
Epoch 0, Step 5, avg loss = 37.7145, acc0 = 0.12, acc1 = 0.09, acc2 = 0.09, acc3 = 0.09
Epoch 0, Step 6, avg loss = 12.0404, acc0 = 0.16, acc1 = 0.07, acc2 = 0.12, acc3 = 0.12
Epoch 0, Step 7, avg loss = 10.0022, acc0 = 0.08, acc1 = 0.08, acc2 = 0.06, acc3 = 0.16
Epoch 0, Step 8, avg loss = 9.4674, acc0 = 0.23, acc1 = 0.13, acc2 = 0.10, acc3 = 0.09
Epoch 0, Step 9, avg loss = 9.4022, acc0 = 0.12, acc1 = 0.12, acc2 = 0.09, acc3 = 0.14
Epoch 0, Step 10, avg loss = 9.2103, acc0 = 0.18, acc1 = 0.17, acc2 = 0.12, acc3 = 0.13
Epoch 0, Step 11, avg loss = 9.0

Epoch 2, Step 23, avg loss = 2.0297, acc0 = 0.98, acc1 = 0.83, acc2 = 0.80, acc3 = 0.81
Epoch 2, Step 24, avg loss = 2.3212, acc0 = 0.98, acc1 = 0.84, acc2 = 0.73, acc3 = 0.77
Epoch 2, Step 25, avg loss = 2.1616, acc0 = 0.97, acc1 = 0.83, acc2 = 0.79, acc3 = 0.78
Epoch 2, Step 26, avg loss = 1.8865, acc0 = 0.92, acc1 = 0.83, acc2 = 0.77, acc3 = 0.88
Epoch 2, Step 27, avg loss = 2.0519, acc0 = 0.98, acc1 = 0.86, acc2 = 0.75, acc3 = 0.82
Epoch 2, Step 28, avg loss = 2.0843, acc0 = 0.98, acc1 = 0.86, acc2 = 0.73, acc3 = 0.88
Epoch 2, Step 29, avg loss = 1.8270, acc0 = 0.95, acc1 = 0.91, acc2 = 0.75, acc3 = 0.80
Epoch 2, Step 30, avg loss = 2.0216, acc0 = 0.95, acc1 = 0.90, acc2 = 0.79, acc3 = 0.77
Epoch 2, Step 31, avg loss = 1.9043, acc0 = 0.97, acc1 = 0.90, acc2 = 0.76, acc3 = 0.80
Epoch 2, Step 32, avg loss = 2.1340, acc0 = 0.95, acc1 = 0.79, acc2 = 0.77, acc3 = 0.79
Epoch 2, Step 33, avg loss = 1.7520, acc0 = 0.97, acc1 = 0.84, acc2 = 0.81, acc3 = 0.84
Epoch 2, Step 34, avg loss = 1.9

Epoch 5, Step 11, avg loss = 0.5877, acc0 = 0.99, acc1 = 0.97, acc2 = 0.95, acc3 = 0.96
Epoch 5, Step 12, avg loss = 0.4825, acc0 = 1.00, acc1 = 0.96, acc2 = 0.92, acc3 = 0.98
Epoch 5, Step 13, avg loss = 0.7422, acc0 = 1.00, acc1 = 0.95, acc2 = 0.95, acc3 = 0.93
Epoch 5, Step 14, avg loss = 0.6371, acc0 = 1.00, acc1 = 0.95, acc2 = 0.93, acc3 = 0.95
Epoch 5, Step 15, avg loss = 0.5802, acc0 = 1.00, acc1 = 0.97, acc2 = 0.94, acc3 = 0.92
Epoch 5, Step 16, avg loss = 0.3728, acc0 = 1.00, acc1 = 0.99, acc2 = 0.93, acc3 = 0.96
Epoch 5, Step 17, avg loss = 0.5256, acc0 = 1.00, acc1 = 0.94, acc2 = 0.95, acc3 = 0.98
Epoch 5, Step 18, avg loss = 0.3810, acc0 = 1.00, acc1 = 0.97, acc2 = 0.96, acc3 = 0.98
Epoch 5, Step 19, avg loss = 0.3947, acc0 = 1.00, acc1 = 0.99, acc2 = 0.91, acc3 = 0.96
Epoch 5, Step 20, avg loss = 0.3441, acc0 = 1.00, acc1 = 0.96, acc2 = 0.97, acc3 = 0.95
Epoch 5, Step 21, avg loss = 0.6281, acc0 = 1.00, acc1 = 0.98, acc2 = 0.96, acc3 = 0.95
Epoch 5, Step 22, avg loss = 0.7

## 8 预测阶段
训练结束之后，我们就可以进入预测阶段。
> 首先我们同样要定义一个用来预测的函数infer()。
> 这个函数的主要功能就是载入我们训练好的模型，传入预测用的图片，来执行预测。
> infer()函数具体应该怎么写呢？如下所示：
> <img src="images_jupyter/infer_1.png" width="600"><br/>

In [14]:
def infer(model_path, data_path, shape):
    """
    Infer by using provided test data.
    """

    place = fluid.CPUPlace()
    exe = fluid.Executor(place)
    inference_scope = fluid.core.Scope()
    with fluid.scope_guard(inference_scope):
        [inference_program, feed_target_names, fetch_targets] = (
            fluid.io.load_inference_model(model_path, exe))

        test_data = data_reader(data_path, shape, batch_size=1).__next__()
        test_x = np.array(list(map(lambda x: x[0], test_data))).astype("float32")
        test_y0 = test_data[0][1]
        test_y1 = test_data[0][2]
        test_y2 = test_data[0][3]
        test_y3 = test_data[0][4]

        results = exe.run(inference_program,
                          feed={feed_target_names[0]: np.array(test_x)},
                          fetch_list=fetch_targets)

        print("infer results: ", results[0].argmax(), results[1].argmax(), results[2].argmax(),
                     results[3].argmax())
        print("ground truth: ", int(test_y0), int(test_y1), int(test_y2), int(test_y3))

调用infer()函数，执行预测。
预测输出形式如下：
<img src="images_jupyter/result_1.png" width="600"><br/>

In [17]:
infer(model_path, test_path, image_shape)  #利用训练好的数据进行预测

infer results:  4 5 1 2
ground truth:  4 5 1 2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


## 9 总结
本次实验我们学习了卷积神经网络的原理及组成。并且用paddlePaddle框架搭建了一个基于卷积神经网络的分类器，用于识别验证码，我们课上实现的是4位验证码的识别，还有一个6位验证码的识别交给大家自己动手来做。后续我们还会有计算机视觉方面的实验项目，等待着大家一起来学习分享。